# [모듈 2.1] NCF 훈련 파이프라인

이 노트북은 아래와 같은 목차로 진행 됩니다. 전체를 모두 실행시에 완료 시간은 **약 15분** 소요 됩니다.

- 0. 배포를 위한 모델 아티펙트 형태
- 1. 환경 설정
- 2. 패키징 코드
- 3. 세이지 메이커 파이프라인 생성
    - 모델 훈련 스텝
    - 모델 아티펙트 리패키징 람다 스텝 정의
    - 모델 등록 스텝
- 4.모델 빌딩 파이프라인 정의 및 실행
- 5.Pipeline 캐싱 및 파라미터 이용한 실행
    
---

# 0. 배포를 위한 모델 아티펙트 형태
- 훈련을 통해서 만들어진 모델 아티펙트 (model.tar.gz) 는 기본적으로 모델 가중치 파일만이 존재합니다. (예: 파이토치는 *.pth 파일) 배포 및 서빙을 위해서는 "의존성 코드 (예: inference.py)" 가 필요한데요, 이를 아래와 같은 폴더 구조로 넣어주고 model.tar.gz 를 만들어야 합니다. 
- 참조 자료: [Getting started with deploying real-time models on Amazon SageMaker](https://aws.amazon.com/blogs/machine-learning/getting-started-with-deploying-real-time-models-on-amazon-sagemaker/)

```
# Pytorch
model.tar.gz/
             |- model.pth
             |- code/
                     |- inference.py
                     |- requirements.txt # only for versions 1.3.1 and higher
                     
# Tensorflow
model.tar.gz/
             |--[model_version_number]/
                                       |--variables
                                       |--saved_model.pb
            code/
                |--inference.py
                |--requirements.txt
```

# 1.환경 설정 



In [ ]:
%load_ext autoreload
%autoreload 2

# src 폴더 경로 설정
import sys
sys.path.append('./src')

In [ ]:
import boto3
import sagemaker
import pandas as pd

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker.Session().default_bucket()

print("bucket: ", bucket)
print("role: ", role)

sm_client = boto3.client('sagemaker', region_name=region)


In [ ]:
%store -r model_package_group_name
%store -r s3_input_data_uri
%store -r bucket
%store -r project_prefix

In [ ]:
print("s3_input_data_uri: \n", s3_input_data_uri)
print("bucket: \n", bucket)
print("project_prefix: \n", project_prefix)
print("model_package_group_name: \n", model_package_group_name)

---
# 2. 패키징 코드
- 패키징할 폴더 생성
- 추론 코드의 폴더 지정 후에 패키징 폴더에 복사
- 패키징 폴더를 source.tar.gz 압축 후에 S3에 업로딩
- 



## 2.1. src 폴더를 source.tar.gz 로 압축

In [ ]:
import os

package_dir = 'code_pkg'
os.makedirs(package_dir, exist_ok=True)


In [ ]:
code_dir = '../src'

In [ ]:
%%sh -s {package_dir} {code_dir}

package_dir=$1
code_dir=$2

cd $package_dir # 폴더 생성
echo $PWD
rm -rf ./*
cp -r $code_dir/*.py  .  # src py 모두 카피
cp -r $code_dir/*.txt  .  # src 파일 모두 카피
cp -r $code_dir/*.json  .  # json 파일 모두 카피
tar -czvf source.tar.gz * # model.tar.gz 파일 생성

## 2.2. S3 에 업로드

In [ ]:
import sagemaker
sagemaker_session  = sagemaker.session.Session()
prefix='ncf/code'
bucket = sagemaker.session.Session().default_bucket()
print("bucket: ", bucket)

source_path = os.path.join(package_dir, 'source.tar.gz')
source_artifact = sagemaker_session.upload_data(source_path, bucket, prefix)
print("source_artifact: \n", source_artifact)

# 3. 세이지 메이커 파이프라인 생성

## 3.1. 모델 빌딩 파이프라인 변수 및 세션 생성

파이프라인에 인자로 넘길 변수는 아래 크게 3가지 종류가 있습니다.
- 모델 레지스트리에 모델 등록시에 모델 승인 상태 값    


In [ ]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)

# 입력 데이터
s3_data_loc = ParameterString(
    name="InputData",
    default_value=s3_input_data_uri,
)


model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)


### 3.1.1 로컬 모드 설정 
- 로컬 모드 사용을 위해서는 Estimator, Pipeline() 오브젝트 생성시에 인자로서 sagemaker_session 에 LocalPipelineSession() 를 할당해야 합니다.
- 모델 훈련 스텝은 로컬 모드가 가능합니다. 
    - 람다 스텝, 모델 등록 스텝은 지원을 하지 않음.
- Tip : 노트북 하단에서 Pipeline() 를 선언시에 steps 부분을 주석 처리하면서 사용하세요.
``` python
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        s3_data_loc,                
        model_approval_status,        
    ],
    sagemaker_session=pipeline_session,
    steps=[step_train],    # 로컬 모드 사용시
#   steps=[step_train, step_repackage_lambda, step_model_registration],
```

In [ ]:
# LOCAL_MODE = True # 로컬 모드시 사용
LOCAL_MODE = False # 클라우드 모드시 사용
if LOCAL_MODE:
    from sagemaker.workflow.pipeline_context import LocalPipelineSession
    pipeline_session = LocalPipelineSession()
    print("### --> Local Mode")
else:
    from sagemaker.workflow.pipeline_context import PipelineSession
    pipeline_session = PipelineSession()
    print("### --> Cloud Mode")    

### 3.1.2 캐싱 정의
참고: 캐싱 파이프라인 단계: [Caching Pipeline Steps](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/pipelines-caching.html)

In [ ]:
from sagemaker.workflow.steps import CacheConfig

cache_config = CacheConfig(enable_caching=True, 
                           expire_after="1d")

## 3.2. 파이프라인 스텝 단계 정의

### 3.2.1 모델 훈련 스텝

####  하이퍼파라미터 세팅

In [ ]:
host_hyperparameters = {'epochs': 1, 
                       'lr': 0.001,
                       'batch_size': 256,
                       'top_k' : 10,
                       'dropout' : 0.0,
                       'factor_num' : 32,
                       'num_layers' : 3,
                       'num_ng' : 4,
                       'test_num_ng' : 99,                   
                    }  

#### 훈련 메트릭을 CloudWatch 에서 보기
- 개발자 가이드
    - [Monitor and Analyze Training Jobs Using Amazon CloudWatch ](https://docs.amazonaws.cn/en_us/sagemaker/latest/dg/training-metrics.html#define-train-metrics)

In [ ]:
metric_definitions=[
       {'Name': 'HR', 'Regex': 'HR=(.*?);'},
       {'Name': 'NDCG', 'Regex': 'NDCG=(.*?);'},
       {'Name': 'Loss', 'Regex': 'Loss=(.*?);'}        
    ]


####  Estimator 생성

Estimator 생성시에 인자가 필요 합니다. 주요한 인자만 보겠습니다.


In [ ]:
from sagemaker.pytorch import PyTorch

estimator_output_path = f's3://{bucket}/{project_prefix}/training_jobs'
print("estimator_output_path: \n", estimator_output_path)


instance_type = 'ml.p3.2xlarge'
instance_count = 1

host_estimator = PyTorch(
    entry_point="train.py",    
    source_dir='src',    
    role=role,
    output_path = estimator_output_path,    
    framework_version='1.12.1',
    py_version='py38',
    disable_profiler = True,
    instance_count=instance_count,
    instance_type=instance_type,
    session = pipeline_session,
    hyperparameters=host_hyperparameters,
    metric_definitions = metric_definitions
    
)


#### 모델 훈련 스탭 생성


In [ ]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


step_train = TrainingStep(
    name= "NCF-Training",
    estimator=host_estimator,
    inputs={
        "train": TrainingInput(
            s3_data= s3_data_loc
        ),
        "test": TrainingInput(
            s3_data= s3_data_loc
        ),        
    },
    cache_config = cache_config, # 캐시 정의     
)

### 3.2.2 모델 아티펙트 리패키징 람다 스텝 정의
- 하단에서 파이프라인을 실행한 후에 람다 스텝의 디버깅을 하기 위해서는 람다 콘솔에 가셔서 해당 람다 함수의 실행 로그를 확인하시면 됩니다.
    - [람다 콘솔](http://console.aws.amazon.com/lambda/)

####  람다 Role 가져오기
- 아래 MLOps-LambdaDeploymentRole 역할은 워크샵 시작시에 "클라우드 포메이션"으로 미리 생성한 역할 입니다.

In [ ]:
import boto3

lambda_role = boto3.client('iam').get_role(RoleName = 'MLOps-LambdaDeploymentRole').get('Role').get('Arn')
print("lambda_role: \n", lambda_role)



#### 람다 스텝 정의

In [ ]:
from datetime import datetime
currentDateAndTime = datetime.now()

currentTime = currentDateAndTime.strftime("%Y-%m-%d-%H-%M-%S")
bucket_prefix = f'ncf/repackage/model/{currentTime}'
print("bucket prefix: \n", bucket_prefix)

from sagemaker.lambda_helper import Lambda
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)

from pathlib import Path

BASE_DIR = Path.cwd()
# BASE_DIR = os.path.dirname(os.path.realpath(__file__))
repackage_lambda_script_path = f'{BASE_DIR}/src/iam_repackage_model_artifact.py'
print("repackage_lambda_script_path: \n", repackage_lambda_script_path)

function_name = "sagemaker-lambda-step-repackage-model-artifact"

print("function_name: \n", function_name)


In [ ]:
# Lambda helper class can be used to create the Lambda function
func_repackage_model = Lambda(
    function_name=function_name,
    execution_role_arn=lambda_role,
    script=repackage_lambda_script_path,
    handler="iam_repackage_model_artifact.lambda_handler",
)

output_param_1 = LambdaOutput(output_name="statusCode", output_type=LambdaOutputTypeEnum.String)
output_param_2 = LambdaOutput(output_name="body", output_type=LambdaOutputTypeEnum.String)
output_param_3 = LambdaOutput(output_name="S3_Model_URI", output_type=LambdaOutputTypeEnum.String)

step_repackage_lambda = LambdaStep(
    name="LambdaRepackageStep",
    lambda_func=func_repackage_model,
    inputs={
        "source_path" : source_artifact,
        "model_path": step_train.properties.ModelArtifacts.S3ModelArtifacts,
        "bucket" : bucket,
        "prefix" : bucket_prefix
    },
    outputs=[output_param_1, output_param_2, output_param_3],
)


### 3.2.3 모델 등록 스텝

#### 모델 그룹 생성

- 참고
    - 모델 그룹 릭스팅 API:  [ListModelPackageGroups](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_ListModelPackageGroups.html)
    - 모델 지표 등록: [Model Quality Metrics](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/model-monitor-model-quality-metrics.html)

In [ ]:
model_package_group_name = f"NCFModel"
model_package_group_input_dict = {
 "ModelPackageGroupName" : model_package_group_name,
 "ModelPackageGroupDescription" : "Sample model package group"
}
response = sm_client.list_model_package_groups(NameContains=model_package_group_name)
if len(response['ModelPackageGroupSummaryList']) == 0:
    print("No model group exists")
    print("Create model group")    
    
    create_model_pacakge_group_response = sm_client.create_model_package_group(**model_package_group_input_dict)
    print('ModelPackageGroup Arn : {}'.format(create_model_pacakge_group_response['ModelPackageGroupArn']))    
else:
    print(f"{model_package_group_name} exitss")

#### 모델 등록 스텝 정의

In [ ]:
%%time 

from sagemaker.workflow.model_step import ModelStep


from sagemaker.model import Model

inference_image_uri = f'763104351884.dkr.ecr.{region}.amazonaws.com/pytorch-inference:1.8.1-gpu-py3'

model = Model(
    image_uri=inference_image_uri,
    model_data = step_repackage_lambda.properties.Outputs["S3_Model_URI"],
    role=role,
    sagemaker_session=pipeline_session,
)


register_model_step_args = model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.g4dn.xlarge", "ml.p2.xlarge"],
    transform_instances=["ml.g4dn.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
)

step_model_registration = ModelStep(
   name="RegisterModel",
   step_args=register_model_step_args,
)

# 4.모델 빌딩 파이프라인 정의 및 실행
위에서 정의한 아래의 4개의 스텝으로 파이프라인 정의를 합니다.


In [ ]:
from sagemaker.workflow.pipeline import Pipeline

project_prefix = 'ncf-pipeline-nb-training'

pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        s3_data_loc,                
        model_approval_status,        
    ],
    sagemaker_session=pipeline_session,
#    steps=[step_train],    
    steps=[step_train, step_repackage_lambda, step_model_registration],
#    steps=[step_repackage_lambda, step_model_registration],    

)



In [ ]:
import json

definition = json.loads(pipeline.definition())
# print(" definition : \n", definition)
pipeline.upsert(role_arn=role)
execution = pipeline.start()

### 파이프라인 운영: 파이프라인 대기 및 실행상태 확인

워크플로우의 실행상황을 살펴봅니다. 

실행이 완료될 때까지 기다립니다.

In [ ]:
execution.wait()

실행된 단계들을 리스트업합니다. 파이프라인의 단계실행 서비스에 의해 시작되거나 완료된 단계를 보여줍니다.

In [ ]:
execution.list_steps()

# 5. Pipeline 캐싱 및 파라미터 이용한 실행
캐싱은 2021년 7월 현재 Training, Processing, Transform 의 Step에 적용이 되어 있습니다.

상세 사항은 여기를 확인하세요. --> [캐싱 파이프라인 단계](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/pipelines-caching.html)

In [ ]:
# is_cache = True
is_cache = False

In [ ]:
%%time 

from IPython.display import display as dp
import time

if is_cache:
    execution = pipeline.start(
        parameters=dict(
            # model2eval2threshold=0.8,
        )
    )    
    
    # execution = pipeline.start()
    execution.wait()

In [ ]:
if is_cache:
    dp(execution.list_steps())

# 6. SageMaker Studio 에서 파이프라인 보기
스튜디오를 사용하기 위해서는 아래 링크를 참조하여, 로그인하시기 바랍니다.
- [Amazon SageMaker Studio](https://docs.aws.amazon.com/sagemaker/latest/dg/studio.html)

## 6.1. 모델 훈련 파이프라인 실행 내용 보기

![studio_pipeline_train.png](img/studio_pipeline_train.png)

## 6.2. 모델 훈련 파이프라인 단계 별 실행 결과 보기

![pipeline_graph.png](img/pipeline_graph.png)

## 6.3. 모델 레지스트리 
- 생성된 "NCFModel" 을 확인하세요.
    - 나머지 모델 그룹은 저자가 생성한 것이기에 무시하시기 바랍니다.

![model_registry.png](img/model_registry.png)

## 6.4. 모델 그룹 버전 정보
- 처음 생성한 버전이 보입니다.

![model_group_info.png](img/model_group_info.png)

## 6.5. 모델 버전의 등록된 컨테이너 정보
- 6.4 그림의 Version - 1 을 클릭하고 Settings 탭을 클릭 후에 스크롤 하면 아래의 컨테이너 정보가 보입니다.
- 아래 정보는 모델 등록 스텝에서 우리가 등록한 2가지 중요한 정보 입니다.
    - 추론 도커 이미지 주소 
    - 리패키징한 모델 아티펙트 주소 

![container_info.png](img/container_info.png)